## Savu Reconstruction

Reconstruction was performed with Savu 4.0.

Please cite:

1. Wadeson, N and Basham, M. Savu: A Python-based, MPI Framework for Simultaneous Processing of Multiple, N-dimensional, Large Tomography Datasets. *arXiv* **1610.08015**. (2016)
2. Wadeson, N, Verschoyle, J, Kazantsev, D., et al. DiamondLightSource/Savu: Version 4.0. *Zenodo*. (2021) [DOI](https://doi.org/10.5281/zenodo.5095360) 10.5281/zenodo.5095360.
3. The HDF Group. Hierarchical Data Format, version 5. (1997-2016)
4. Walker, D and Dongarra, J. MPI: a standard message passing interface. *Supercomputer* **12**: 56-68. (1996)
5. Van Aarle, W, Palenstijn, W, Cant, J, et al. Fast and flexible X-ray tomography using the ASTRA toolbox. *Optics express* **24(22)**: 25129-25147. (2016)
6. Van Aarle, W, Palenstijn, W, De Beenhouwer, J, et al. The ASTRA Toolbox: A platform for advanced algorithm development in electron tomography. *Ultramicroscopy* **157**: 35-47. (2015)
7. Palenstijn, WJ and Batenburg, KJ and Sijbers, J. Performance improvements for iterative electron tomography reconstruction using graphics processing units (GPUs). *JSB* **176(2)** 250-253. (2011)


For more information, please see https://savu.readthedocs.io/en/latest/

<b>Note</b>

Savu was developed by the X-ray tomography community, so the conventional relationship between grey values and density is reversed compared to EM, i.e., the colours are reversed. The visualisations in this report have had their colours inverted to match the EM convention.

In [ ]:
from scipy.ndimage import gaussian_filter as gf

In [ ]:
def print_recon_algo():
    """Print Reconstruction algorithm used"""
    savu_yaml = f"{os.getcwd()}/{proj_name}_savurecon.yaml"

    try:
        with open(savu_yaml, "r") as f:
            output = yaml.load(f.read(), Loader=yaml.FullLoader)
            algorithm = output["Savu"]["setup"]["algorithm"]
    except FileNotFoundError:
        print(f"{savu_yaml} not found, cannot find algorithm used")

    printmd(f"Reconstruction algorithm used: {algorithm}")


print_recon_algo()

In [ ]:
def get_central_slices(img_data):
    # Find central slices
    z_central = int(img_data.shape[0] / 2)
    y_central = int(img_data.shape[1] / 2)
    x_central = int(img_data.shape[2] / 2)

    # Calculate number of slices to add
    zxy_add = (np.array(img_data.shape) * 0.0025).astype(int)
    zxy_add[zxy_add < 1] = 1

    # xy, yz, xz image data
    xy_data = img_data[z_central - zxy_add[0] : z_central + zxy_add[0], :, :].sum(
        axis=0
    )
    yz_data = img_data[:, x_central - zxy_add[1] : x_central + zxy_add[1], :].sum(
        axis=1
    )
    xz_data = img_data[:, :, y_central - zxy_add[2] : y_central + zxy_add[2]].sum(
        axis=2
    )

    return [xy_data, yz_data, xz_data]


def show_tomogram(tomo_fname):
    """tomo_fname can be 3-D .mrc, 3-D .tiff, or stack of 2-D tiffs"""

    # Import image data
    if tomo_fname.endswith(".mrc"):
        with mrcfile.open(tomo_fname) as mrc:
            img_data = mrc.data

    elif os.path.isdir(tomo_fname):
        volume = []
        for xy_slice in os.listdir(tomo_fname):
            volume.append(tifffile.imread(f"{tomo_fname}/{xy_slice}").data)
        img_data = np.stack(volume)

    elif tomo_fname.endswith(".tif"):
        img_data = tifffile.imread(tomo_fname)

    else:
        raise ValueError(
            "Tomogram cannot be read. Please use a 3D tiff, mrc or stack of 2D Tiffs"
        )

    xy_data, yz_data, xz_data = get_central_slices(img_data)
    data_flattened = np.hstack(
        (xy_data.flatten(), yz_data.flatten(), xz_data.flatten())
    )

    # Show central slices in xy, yz, xz
    plt.figure(figsize=(10, 3.5))
    plt.suptitle(os.path.basename(tomo_fname))
    titles = ["x-y", "y-z", "x-z"]

    for i, data in enumerate([xy_data, yz_data, xz_data]):
        plt.subplot(1, 3, i + 1)
        plt.imshow(
            data,
            cmap="Greys",
            vmin=np.percentile(data_flattened[data_flattened > 0], 10),
            vmax=np.percentile(data_flattened[data_flattened > 0], 95),
        )
        plt.title(titles[i])

    plt.tight_layout()

In [ ]:
def get_tomogram_filenames():
    # Get Savu mdout
    try:
        savu_mdout = f"{os.getcwd()}/{proj_name}_savurecon_mdout.yaml"
    except FileNotFoundError:
        print(
            f"{proj_name}_savurecon_mdout.yaml not found, cannot find tomogram filenames."
        )

    with open(savu_mdout, "r") as f:
        output = yaml.load(f.read(), Loader=yaml.FullLoader)
        tomogram_filenames = output["tomogram"].values()

    return tomogram_filenames


tomogram_filenames = get_tomogram_filenames()

#### Tomogram thumbnails

In [ ]:
# Show thumbnails of the tomograms

for tomogram in tomogram_filenames:
    show_tomogram(tomogram)